넷플릭스 콘텐츠 분석하기
=================

안녕하세요, 킹래빗입니다🐰    
데이터 분석으로 처음 글을 적어보는데요,     
    
다들 많이 보시는 넷플릭스 영화 및 TV 콘텐츠를 분석해 본 내용입니다🎬🍿    
데이터는 kaggle의 데이터셋을 이용해서 **미국 넷플릭스 데이터**라고 보시면 될 것 같아요.       

> 데이터와 분석 노트북은 깃허브 리포지토리에서 확인하실 수 있습니다.     
> + [데이터셋 바로가기][link]
> + [리포지토리 바로가기][link_repository]

[link]: https://www.kaggle.com/shivamb/netflix-shows "netflix-shows-dataset"
[link_repository]: https://github.com/king-rabbit

## 1. 데이터 불러오기
---------------------
필요한 라이브러리와 데이터를 불러오고 데이터 형식 등을 확인해봅니다.    
**참고로 pyecharts는 0.5.11버전입니다.**

In [2]:
import warnings
import pandas as pd
import numpy as np
from pyecharts import Bar, Line, Pie, Map
warnings.filterwarnings(action='ignore')

In [3]:
#데이터 불러오기
data = pd.read_csv('netflix_titles.csv')

* 드라마/영화 콘텐츠에는 고유번호가 지정되어 있습니다(show_id 컬럼)
* type에서는 영화인지 TV쇼인지를 구분합니다
* 출연진(cast)과 국가(country, 두개 이상인 경우)는 comma(,)로 구분되어 있습니다
* 콘텐츠 길이(duration)의 경우 영화면 분 단위로, TV쇼이면 시즌 단위로 표기됩니다
* 장르는 listed_in 컬럼이며, 두개 이상의 장르에 속한 경우 comma(,)로 구분됩니다.

In [6]:
data.head(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


## 2. 어느 나라 콘텐츠가 많을까
------------------
아무래도 나라별로 콘텐츠 개수에 차이가 나겠죠. 국가별 콘텐츠 개수가 어떻게 되는지 확인하기 위해 먼저 country 컬럼을 전처리해 줍니다.    
comma를 기준으로 country 컬럼을 구분해 준 뒤 국가별로 영화/TV콘텐츠 개수를 집계한 데이터프레임을 만드는 방식입니다.

In [4]:
countries = []
for i in range(len(data)):
    if type(data.loc[i, 'country']) == str:
        data.loc[i, 'country'] = data.loc[i, 'country'].replace("\"", "")
        countries_content = data.loc[i, 'country'].split(', ')
        for country in countries_content:
            countries.append(country)
    else:
        pass

countries = list(set(countries))

In [5]:
countries.remove('') #아무것도 없는 문자열 제거
countires_re = [ country.replace(',', '') for country in countries ]
countires_re = list(set(countires_re)) # 국가명 리스트 생성

In [11]:
countries_cnt = []
for country in countires_re:
    cnt = 0
    # 국가별로 영화/TV콘텐츠 개수를 셉니다. 영화의 country컬럼에 해당 국가명이 포함되면 1을 추가하는 방식입니다.
    for i in range(len(data)):  
        if type(data.loc[i, 'country']) == str and data.loc[i, 'country'].__contains__(country):
            cnt +=1
    countries_cnt.append(cnt) 

# count를 기준으로 정렬합니다.
countries_movie_number = pd.DataFrame({'country':countires_re, 'count':countries_cnt}).sort_values(by='count', ascending=False)

> 데이터 프레임을 만든 후에는 이를 pyecharts를 이용해 바 그래프로 시각화해 봅니다.  
* 콘텐츠 수 상위 10개 국가만 시각화했습니다.
* 먼저 Bar 그래프 객체를 생성한 뒤 테마를 설정하고 그래프 이름, x축 및 y축 데이터 등을 설정하는 방식입니다.
* 그래프 객체 설정 시 전체 그래프의 크기(여기서는 width만 설정함)를 정할 수 있습니다.
* x축의 폰트크기는 9입니다.
* label_text_size에서는 그래프 라벨의 폰트크기를 정합니다.
* is_datazoom_show를 False로 설정해 줌이 불가능하게 했습니다.
* 따로 설정하지는 않았는데 bar.use_theme()으로 테마를 설정할 수 있습니다.

In [59]:
bar = Bar("국가별 넷플릭스 콘텐츠 개수",
         width=900)
bar.add("", 
        countries_movie_number['country'][:10], 
        countries_movie_number['count'][:10],
        xaxis_label_textsize=9,
        xaxis_interval=0,
        label_text_size=10,
        is_label_show=True, 
        is_datazoom_show=False)
bar

역시 미국🗽의 콘텐츠 양이 압도적입니다. 다음은 발리우드로 유명한 인도인데 2위임에도 미국과 3.5배 정도 차이가 납니다.    
한국은 231개로 독일과 공동 8위입니다. 멕시코가 169개로 10위를 차지했네요.

### 3. 연도별 콘텐츠 개수 분석
----------------------------
몇 년 전만 해도 넷플릭스에서 볼 게 많지 않다는 평가가 많았는데요.     
최근에는 콘텐츠 양도 절대적으로 많아졌고 오리지널 시리즈도 많이 제작되고 있어서 더이상 이런 평가는 들리지 않는 것 같습니다.    
넷플릭스가 매년 콘텐츠를 얼마나 늘려왔는지 한번 확인해볼까요.      
콘텐츠별로 넷플릭스에 추가된 날짜는 date_added 컬럼에 저장되어 있습니다. 여기서 연도만 date_added_year 컬럼으로 저장해 준 뒤 시각화해보았습니다.

In [23]:
for i in range(len(data)):
    if type(data.loc[i, 'date_added']) == str:
        data.loc[i, 'date_added_year'] = data.loc[i, 'date_added'][-4:]

In [24]:
contents_added_year = pd.DataFrame(data['date_added_year'].value_counts()).reset_index().rename(columns={'index':'year', 'date_added_year':'count'}).sort_values(by='year')

In [58]:
bar = Bar("연도별 추가된 콘텐츠 수",
         width=900)
bar.add("", 
        contents_added_year['year'], 
        contents_added_year['count'],
        xaxis_label_textsize=9,
        xaxis_interval=0,
        label_text_size=10,
        is_label_show=True, 
        is_stack=True,
        is_datazoom_show=False)
bar

### 4. 유형/장르 분석
-----------------------
어떤 유형/장르의 콘텐츠가 넷플릭스에 많은지도 개인적으로 궁금했던 부분입니다.    
유형은 TV쇼인지 영화인지 구분하는 걸 의미해서 분석이 용이합니다.     
반면에 영화와 TV쇼의 장르 구분이 별개로 되어 있고 한 콘텐츠가 여러 장르로 구분되어 있어서 구분이 명확하지 않긴 하지만, 일단 주어진 데이터로 돌려봤습니다.
    
#### 4-1. 유형 분석
영화와 TV쇼 중 어느 쪽 비중이 큰지를 파이차트로 시각화했습니다.    
영화가 거의 70%에 육박하네요. TV쇼도 많다고 생각했는데, 아직은 영화의 비중이 높군요.

In [53]:
pie = Pie("유형별 넷플릭스 콘텐츠 비중")
pie.add("", data['type'].value_counts().index, data['type'].value_counts().values, is_label_show=True)
pie

#### 4-2. 장르 분석
다음은 장르분석입니다. 국가별 콘텐츠 개수 분석과 마찬가지로 장르별로 콘텐츠 개수를 집계해줍니다.

In [29]:
categories = []

for i in range(len(data)):
    cats = data.loc[i, 'listed_in'].split(', ')
    for cat in cats:
        categories.append(cat)

categories = set(categories)

In [30]:
show_ids = []
categories_list = []

for i in range(len(data)):
    cats = data.loc[i, 'listed_in'].split(', ')
    for cat in cats:
        show_ids.append(data.loc[i, 'show_id'])
        categories_list.append(cat)

In [31]:
show_category = pd.DataFrame({'show_id':show_ids, 'category':categories_list})

In [57]:
bar = Bar("장르별 콘텐츠 개수",
         width=900)
bar.add("", 
        show_category.category.value_counts()[:10].keys(), 
        show_category.category.value_counts()[:10].values,
        bar_category_gap='10%',
        xaxis_type='category',
        xaxis_label_textsize=9,
        xaxis_interval=0,
        label_text_size=10,
        is_label_show=True, 
        is_datazoom_show=False)
bar

미국 외 다른 나라의 영화/TV쇼를 의미하는 International Movies 장르의 콘텐츠가 가장 많네요. International TV Show도 1351개로 4위이구요. 넷플릭스가 글로벌한 서비스이다보니 자연스레 나타난 결과인 것 같습니다.    
드라마가 2위인데 1위와 별로 차이가 안나는 게 오히려 의외입니다. 3위인 코미디와도 개수 차이가 크고요.    
다큐멘터리가 액션 영화보다 약간 더 많은 점, TV 드라마가 이 두 장르보다 적은 것도 개인적으로는 의외입니다🤔   
넷플릭스에서 다큐멘터리가 생각보다 인기라는 얘기를 들었는데 이런 점이 반영된 것일 수도 있겠네요.
    
-----------------------------
    
여기까지 넷플릭스 콘텐츠 데이터를 분석해보았는데요, 생각보다 글이 길어져서 2편에서 마저 적으려고 합니다😅      
읽어주셔서 감사하고 궁금하신 부분 있으면 댓글로 달아주세요!    
